In [25]:
import torch
from models import PatchTST, Autoformer, Transformer, DLinear
from data_provider import data_factory
import os
import numpy as np
import pickle
import warnings

warnings.filterwarnings("ignore")


def getDataset(args):
    data_set, data_loader = data_factory.data_provider(args, "pred")
    return data_set, data_loader


def loadArgs(model_id: str, data_path='AAPL.csv'):
    path = getArgsPath(model_id)
    with open(path,
              'rb') as f:
        args = pickle.load(f)
        args.root_path = './dataset/'
        args.data_path = data_path
        args.batch_size = 1
        args.seq_len = 336
        args.label_len = 96
        args.pred_len = 96
        return args


def getModelPath(model_id: str):
    return "results/" + model_id + "/checkpoints/checkpoint.pth"


def getArgsPath(model_id: str):
    return "results/" + model_id + "/args.pkl"


def getModel(model_id: str):
    model_path = getModelPath(model_id)

    # Load model
    args = loadArgs(model_id)
    model = None

    print(model_id)
    print(model_id)
    print(model_id)

    # if model id includes model name
    if 'PatchTST' in model_id:
        model = PatchTST.Model(args).float()
    elif 'Autoformer' in model_id:
        model = Autoformer.Model(args).float()
    elif 'Transformer' in model_id:
        model = Transformer.Model(args).float()
    elif 'DLinear' in model_id:
        model = DLinear.Model(args).float()

    path = os.path.join(model_path)
    model.load_state_dict(torch.load(
        path, map_location=torch.device('cpu')))

    return model


def predict(model_id: str, data):
    model = getModel(model_id)

    device = torch.device('cpu')

    (batch_x, batch_y, batch_x_mark, batch_y_mark) = data

    print("batch_y shape before: ", batch_y.shape)
    batch_x = batch_x.reshape(1, batch_x.shape[1], batch_x.shape[2])
    batch_y = batch_y.reshape(1, batch_y.shape[1], batch_y.shape[2])

    batch_x_mark = batch_x_mark.reshape(
        1, batch_x_mark.shape[1], batch_x_mark.shape[2])
    batch_y_mark = batch_y_mark.reshape(
        1, batch_y_mark.shape[1], batch_y_mark.shape[2])

    batch_x = batch_x.float().to(torch.device('cpu'))
    batch_y = batch_y.float()
    print("batch_y shape: ", batch_y.shape)
    batch_x_mark = batch_x_mark.float().to(torch.device('cpu'))
    batch_y_mark = batch_y_mark.float().to(torch.device('cpu'))

    if 'Linear' in model_id or 'TST' in model_id:
        outputs = model(batch_x)
    else:
        # decoder input
        dec_inp = torch.zeros_like(
            batch_y[:, -96:, :]).float()
        dec_inp = torch.cat(
            [batch_y[:, :48, :], dec_inp], dim=1).float().to(torch.device('cpu'))

        outputs = model(
            batch_x, batch_x_mark, dec_inp, batch_y_mark)

    # Predict
    pred = outputs.detach().cpu().numpy()  # .squeeze()
    return pred

In [1]:

import pandas as pd
import matplotlib.pyplot as plt
from predict import predict
import os

root_path = 'dataset'
data_path = 'AAPL.csv'
input_size = 336
output_size = 96

# Load data
data = pd.read_csv(os.path.join(root_path, data_path))

# only use input_size + output_size data
data = data.iloc[-(input_size + output_size):]

models = ["PatchTST", "Autoformer", "Transformer", "DLinear"]

plt.figure(figsize=(20, 10))

for i, model in enumerate(models):
    # predict
    model_id = model + "_AAPL_336_96"
    pred = predict(model_id, data)
    input = data.iloc[:input_size]
    output = data.iloc[input_size:]
    print(input)

    # # plot the results
    # fig, ax = plt.subplots(len(models), i + 1)
    # ax[0].plot(np.arange(input_size), input_plot, label='input')
    # ax[0].plot(np.arange(input_size, input_size + output_size),
    #            output_plot, label='output')
    # ax[0].plot(np.arange(input_size, input_size + output_size),
    #            pred.squeeze(), label='prediction')
    # ax[0].legend()
# plt.show()

Use CPU
>>>>>>>predicting : PatchTST_AAPL_336_96<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
pred 1


RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.